In [64]:
import smtplib, ssl, email, mimetypes
from email.message import EmailMessage
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import csv

In [52]:
sender_email = "fama.shares@gmail.com"
#receiver_email = "maxludwig3@t-online.de"
port = 465
password = "www.pornhub.com"

In [78]:
Kapital = 10300.54
receiver_email = "brandmueller.fabian@gmail.com"
aktien_preis_anzahl_empfehlung_list = [["aktie1", 8.1, 3], ["aktie2", 7.8, 5], ["aktie3", 5.9, 13]]
aktien_anzahl_list = [["aktie1", 3], ["aktie2", 1], ["aktie3", 5]]

send_email_sell(receiver_email, Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list)

In [71]:
def send_email_sell(receiver_email, Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list):
    message = build_email_buy(Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list)
    write_csv(aktien_preis_anzahl_empfehlung_list)
    
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = "FAMA.SHARES - Neue Verkaufsempfehlung"
    msg.attach(MIMEText(message, "plain"))
    msg = attach_csv(msg)
    msg = msg.as_string()
    
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

In [62]:
def attach_csv(msg):
    fileToSend = "aktien.csv"
    
    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(fileToSend, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "audio":
        fp = open(fileToSend, "rb")
        attachment = MIMEAudio(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)
    
    
    
    
    
    
    
    
    
    
    
    
    #with open(filename, "rb") as attachment:
    #    part = MIMEText("application", "octet-stream")
    #    part.set_payload(attachment.read())
#
#    encoders.encode_base64(part)
 #   part.add_header(
  #      "Content-Disposition",
  #      f"attachment; filename= {filename}",
  #  )
  #  msg.attach(part)
    return msg

In [76]:
def build_email_buy(Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list):
    aktien_bestand_string = ""
    for aktie in aktien_anzahl_list:
        aktien_bestand_string += aktie[0] + ": " + str(aktie[1]) + "\r\n"
        
    aktien_buy_string = ""
    for aktie in aktien_preis_anzahl_empfehlung_list:
        aktien_buy_string += "-----\n"
        aktien_buy_string += aktie[0] + "\n"
        aktien_buy_string += "Aktueller Wert: " + str(aktie[1]) + " Euro\r\n"
        aktien_buy_string += "Empfohlene Anzahl: " + str(aktie[2]) + " Euro\r\n"
        aktien_buy_string += "-----\r\n"
        
    message = """\
Subject: FAMA.SHARES - Neue Kaufempfehlung
    
Guten Tag,
    
Es wird empfohlen folgende Aktie(n) zu kaufen:
{aktien_buy_string}
    
Falls Sie sich fuer einen Kauf entscheiden, schicken Sie uns bitte die beigefuegt CSV-Datei ausgefuellt zurueck.
    
PS:
Ihr aktuelles Kapital liegt bei {Kapital} Euro
    
Hier ihre aktuellen Aktienbestaende:
{aktien_bestand_string}""".format(aktien_buy_string=aktien_buy_string, Kapital=Kapital, aktien_bestand_string=aktien_bestand_string)
    
    return message

In [77]:
def build_email_sell(Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list):
    aktien_bestand_string = ""
    for aktie in aktien_anzahl_list:
        aktien_bestand_string += aktie[0] + ": " + str(aktie[1]) + "\r\n"
        
    aktien_buy_string = ""
    for aktie in aktien_preis_anzahl_empfehlung_list:
        aktien_buy_string += "-----\n"
        aktien_buy_string += aktie[0] + "\n"
        aktien_buy_string += "Aktueller Wert: " + str(aktie[1]) + " Euro\r\n"
        aktien_buy_string += "Empfohlene Anzahl: " + str(aktie[2]) + " Euro\r\n"
        aktien_buy_string += "-----\r\n"
        
    message = """\
Subject: FAMA.SHARES - Neue Verkaufsempfehlung
    
Guten Tag,
    
Es wird empfohlen folgende Aktie(n) zu verkaufen:
{aktien_buy_string}
    
Falls Sie sich fuer einen Verkauf entscheiden, schicken Sie uns bitte die beigefuegte CSV-Datei ausgefuellt zurueck.
    
PS:
Ihr aktuelles Kapital liegt bei {Kapital} Euro
    
Hier ihre aktuellen Aktienbestaende:
{aktien_bestand_string}""".format(aktien_buy_string=aktien_buy_string, Kapital=Kapital, aktien_bestand_string=aktien_bestand_string)
    
    return message

In [19]:
def write_csv(aktien_preis_anzahl_empfehlung_list):
    with open('aktien.csv', mode='w') as csv_file:
        aktien_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        aktien_writer.writerow(['Aktie', 'Wert', 'Anzahl'])
        for aktie in aktien_preis_anzahl_empfehlung_list:
            aktien_writer.writerow([aktie[0], '', ''])
        


In [42]:
def read_csv():
    aktien_preis_anzahl_list= []
    with open('aktien.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count > 0 and row != []:
                aktien_preis_anzahl_list.append(row)
            line_count += 1
    return aktien_preis_anzahl_list
    

In [43]:
#write_csv(aktien_preis_anzahl_empfehlung_list)
print(read_csv())

[['aktie1', '', ''], ['aktie2', '', ''], ['aktie3', '', '']]


In [9]:
Kapital = 10300.54
aktien_preis_anzahl_empfehlung_list = [["aktie1", 8.1, 3], ["aktie2", 7.8, 5], ["aktie3", 5.9, 13]]
aktien_anzahl_list = [["aktie1", 3], ["aktie2", 1], ["aktie3", 5]]
print(build_email_sell(Kapital, aktien_anzahl_list, aktien_preis_anzahl_empfehlung_list))

NameError: name 'build_email_sell' is not defined

aktie1: 8.1
aktie2: 7.8
aktie3: 5.9

